In [11]:
import json
import requests
from bs4 import BeautifulSoup as BS

In [12]:
# list of urls
d = {}
urls = ['https://en.wikipedia.org/wiki/Alibaba_Group', 
        'https://en.wikipedia.org/wiki/Amazon_(company)', 
        'https://en.wikipedia.org/wiki/EBay', 
        'https://en.wikipedia.org/wiki/Walmart'
       ]
data_to_output = {}
# layout of wiki page is standardized
# text content is all in p tags
for u in urls:
    
    # dictionary to to hold text information by url
    # in case we try and crawl all on a central notebook
    d[u] = []
    wiki_pg = requests.get(u)
    soup = BS(wiki_pg.content, 'lxml')
    toc = str(soup.find(id='toc'))
    
    soup_toc = BS(toc, 'lxml')
    
    toc_num = soup.find_all('span', {"class" : 'tocnumber'})
    toc_data = soup.find_all('span', {"class" : 'toctext'})
    
    d[u] = (toc_num, toc_data)

In [13]:
for u in d.keys():
    
    toc_dict = {}
    toc_num, toc_data = d[u]
    
    # generate list of keys
    keys = {}
    for i in range(len(toc_num)):
        
        if '.' not in toc_num[i].text:
            keys[toc_num[i].text] = toc_data[i].text
    
    # actually add to dictionary now
    for i in range(len(toc_num)):
        
        if '.' not in toc_num[i].text:
            toc_dict[toc_data[i].text] = []
            continue
            
        num = toc_num[i].text.split('.')[0]
        key = keys[num]
        
        if key in toc_dict:
            toc_dict[key].append(toc_data[i].text)
        else:
            toc_dict[key] = [toc_data[i].text]
    
    company_name = u.split("/wiki/")[1]
    data_to_output[company_name] = toc_dict
    print (u)
    print (json.dumps(toc_dict))
    print ('######')


https://en.wikipedia.org/wiki/Alibaba_Group
{"Naming": [], "History": [], "Companies and affiliated entities": ["E-commerce and retail service platforms", "Cloud computing and artificial intelligence technology", "FinTech and online payment platforms", "Entertainment services", "Internet services", "Others"], "Corporate governance": [], "Controversies": ["Gold Supplier membership", "Uranium sales", "Counterfeit items and scams", "Class action on IPO"], "See also": [], "References": [], "External links": []}
######
https://en.wikipedia.org/wiki/Amazon_(company)
{"History": [], "Board of directors": [], "Merchant partnerships": [], "Products and services": [], "Subsidiaries": ["A9.com", "Amazon Maritime", "Audible.com", "Beijing Century Joyo Courier Services", "Brilliance Audio", "ComiXology", "CreateSpace", "Eero", "Goodreads", "Health Navigator", "Lab126", "Kuiper Systems", "Ring", "Shelfari", "Souq", "Twitch", "Whole Foods Market", "Junglee"], "Supply chain": [], "Website": ["Reviews"

In [15]:
#print to file
with open('table_of_content.txt', 'w') as outfile:
    json.dump(data_to_output, outfile)
